# Введение в искусственные нейронные сети
# Урок 3. TensorFlow

## Домашнее задание

  1. Постройте нейронную сеть (берем несложную полносвязную сеть, меняем число слоев, число нейронов, типы активации, тип оптимизатора) на датасете from sklearn.datasets import load_boston. 
  2. Постройте 10-15 вариантов разных нейронных сетей и сведите результаты их работы в таблицу.  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

### 1. Импорты

In [90]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MSE
from tensorflow.keras.layers import Input, Dense, Activation, ReLU, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.utils.generic_utils import get_custom_objects
import tensorflow_addons as tfa
from sklearn.preprocessing import StandardScaler
keras = tf.keras

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
!nvidia-smi

Sat Oct 29 11:49:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 35.2 MB/s 


### 2. Загрузка и подготовка данных

In [5]:
random_state = 69

In [6]:
data = load_boston()
boston = pd.DataFrame(data['data'], columns=data['feature_names'])
boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [67]:
scaler = StandardScaler()
boston = scaler.fit_transform(boston)
boston

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

In [68]:
X_train, X_test, y_train, y_test = train_test_split(boston, data['target'], test_size=0.25, random_state=random_state)

In [70]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)) 
train_dataset = train_dataset.shuffle(buffer_size=X_train.shape[0]).batch(batch_size) 
tf.random.set_seed(1)

### 3. Подготовка архитектур моделей

Будем менять параметры: число слоев, число нейронов, типы активации, тип оптимизатора.

In [91]:
get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})



In [92]:
n_layers = (3, 6)
n_neurons_first_layer = (128, 256)
optimizers = (Adam(learning_rate=0.01), SGD(learning_rate=0.01))
act_func = ['relu', 'leaky-relu']

In [109]:
models = {}
for n_neurons in n_neurons_first_layer:
  for max_layers in n_layers:
    for activation in act_func:            
      inputs = Input(shape=(13, 1), name='boston')
      x = inputs
      for single_layer in range(max_layers):
          x = Dense(n_neurons, activation=activation, name=f'dense_{single_layer}')(x)
      outputs = Dense(1, activation=activation, name='predictions')(x) 
      model = Model(inputs=inputs, outputs=outputs) 
      model_name = f'nn_{n_neurons} neurons_{max_layers + 1} layers_{activation} activation'
      models[model_name] = model


### 4. Обучение моделей

In [110]:
num_epochs = 7
epoch = np.arange(num_epochs+1)
metric = tfa.metrics.r_square.RSquare()
loss_fn = tf.keras.losses.MeanSquaredError()
model_names = []
losses = []

for name, model in models.items():
    for i, i_optimizer in enumerate(optimizers):
        print(f'Обучаем модель {name} с оптимизатором {i_optimizer.get_config()["name"]}')
        model.compile(
        optimizer=i_optimizer,
        loss='mse',
        metrics=[metric],
        )
        for epoch in range(num_epochs): 

            # Итерируем по батчам в датасете
            for step, (x_batch_train, y_batch_train) in enumerate(train_dataset): 
                
                # Откроем GradientTape чтобы записать операции
                # выполняемые во время прямого прохода, включающего автодифференцирование
                with tf.GradientTape() as tape: 
                    # Запустим прямой проход слоя
                    preds = model(x_batch_train) 

                    # Вычислим значение потерь для этого батча
                    loss_value = loss_fn(y_batch_train, preds)

                    # Используем gradient tape для автоматического извлечения градиентов 
                    # обучаемых переменных относительно потерь
                    grads = tape.gradient(loss_value, model.trainable_weights) 
                    g_g = []

                    # пишем логи для сохранения значений градиента и веса по одной цепи 
                    for g_s in grads:
                        # допишем логи значений градиента в зависимости от размера тензора градиента
                        # if len(g_s.numpy().shape) == 1:
                        #     g_g.append(g_s.numpy()[0])
                        if len(g_s.numpy().shape) == 2:
                            g_g.append(g_s.numpy()[0, 0]) 
                # Выполним один шаг градиентного спуска,
                # обновив значение переменных минимизирующих потери
                i_optimizer.apply_gradients(zip(grads, model.trainable_weights)) 

        model_names.append(name + i_optimizer.get_config()["name"])
        losses.append(loss_fn(y_test, model(X_test))) 
results = pd.DataFrame(model_names, columns=['Модель'])
results['MSE на тесте'] = losses

Обучаем модель nn_128 neurons_4 layers_relu activation с оптимизатором Adam
Обучаем модель nn_128 neurons_4 layers_relu activation с оптимизатором SGD
Обучаем модель nn_128 neurons_4 layers_leaky-relu activation с оптимизатором Adam
Обучаем модель nn_128 neurons_4 layers_leaky-relu activation с оптимизатором SGD
Обучаем модель nn_128 neurons_7 layers_relu activation с оптимизатором Adam
Обучаем модель nn_128 neurons_7 layers_relu activation с оптимизатором SGD
Обучаем модель nn_128 neurons_7 layers_leaky-relu activation с оптимизатором Adam
Обучаем модель nn_128 neurons_7 layers_leaky-relu activation с оптимизатором SGD
Обучаем модель nn_256 neurons_4 layers_relu activation с оптимизатором Adam
Обучаем модель nn_256 neurons_4 layers_relu activation с оптимизатором SGD
Обучаем модель nn_256 neurons_4 layers_leaky-relu activation с оптимизатором Adam
Обучаем модель nn_256 neurons_4 layers_leaky-relu activation с оптимизатором SGD
Обучаем модель nn_256 neurons_7 layers_relu activation с о

In [113]:
res = results.copy()
res['MSE на тесте'] = res['MSE на тесте'].apply(lambda x: x.numpy())
res.sort_values(by='MSE на тесте')

,Модель,MSE на тесте
0,nn_128 neurons_4 layers_relu activationAdam,72.307320
4,nn_128 neurons_7 layers_relu activationAdam,74.390144
8,nn_256 neurons_4 layers_relu activationAdam,76.944595
10,nn_256 neurons_4 layers_leaky-relu activationAdam,85.588799
2,nn_128 neurons_4 layers_leaky-relu activationAdam,147.044754
6,nn_128 neurons_7 layers_leaky-relu activationAdam,202.037064
1,nn_128 neurons_4 layers_relu activationSGD,558.081970
5,nn_128 neurons_7 layers_relu activationSGD,558.081970
9,nn_256 neurons_4 layers_relu activationSGD,558.081970
12,nn_256 neurons_7 layers_relu activationAdam,558.081970


Вот так интересно получилось. Хорошие результаты показали оптимизаторы Adam и функции активации ReLU. Лучший результат - на самой простой архитектуре (4 слоя и 128 нейронов).
А если использовать SGD, то градиент сходится хуже (или вообще расходится, и сеть не обучается).
